In [2]:
# Adjust Python path within the notebook
import sys
project_root = '/Users/thangnguyen/Documents/GitHub/project-1-individual-knam2609'
if project_root not in sys.path:
    sys.path.insert(0, project_root)

import scripts

In [3]:
# Create SparkSession
spark = scripts.clean_base.create_spark_session()

24/08/27 21:52:51 WARN Utils: Your hostname, THANGs-MacBook-Air.local resolves to a loopback address: 127.0.0.1; using 100.86.89.69 instead (on interface en0)
24/08/27 21:52:51 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/08/27 21:52:51 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/08/27 21:52:52 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [4]:
# Create directories for plots
scripts.download.make_directories(["../plots/uber/"], ["map"])

In [5]:
import pandas as pd
import geopandas as gpd
import folium

In [6]:
# sf stands for shape file
sf = gpd.read_file("../data/landing/taxi_zones/taxi_zones.shp")
zones = pd.read_csv("../data/landing/taxi_zones.csv")

sf.head()

,OBJECTID,Shape_Leng,Shape_Area,zone,LocationID,borough,geometry
0,1,0.116357,0.000782,Newark Airport,1,EWR,"POLYGON ((933100.918 192536.086, 933091.011 19..."
1,2,0.433470,0.004866,Jamaica Bay,2,Queens,"MULTIPOLYGON (((1033269.244 172126.008, 103343..."
2,3,0.084341,0.000314,Allerton/Pelham Gardens,3,Bronx,"POLYGON ((1026308.77 256767.698, 1026495.593 2..."
3,4,0.043567,0.000112,Alphabet City,4,Manhattan,"POLYGON ((992073.467 203714.076, 992068.667 20..."
4,5,0.092146,0.000498,Arden Heights,5,Staten Island,"POLYGON ((935843.31 144283.336, 936046.565 144..."


In [7]:
# Convert the geometry shape to latitude and longitude
sf['geometry'] = sf['geometry'].to_crs("+proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs")
sf.head()

,OBJECTID,Shape_Leng,Shape_Area,zone,LocationID,borough,geometry
0,1,0.116357,0.000782,Newark Airport,1,EWR,"POLYGON ((-74.18445 40.695, -74.18449 40.6951,..."
1,2,0.433470,0.004866,Jamaica Bay,2,Queens,"MULTIPOLYGON (((-73.82338 40.63899, -73.82277 ..."
2,3,0.084341,0.000314,Allerton/Pelham Gardens,3,Bronx,"POLYGON ((-73.84793 40.87134, -73.84725 40.870..."
3,4,0.043567,0.000112,Alphabet City,4,Manhattan,"POLYGON ((-73.97177 40.72582, -73.97179 40.725..."
4,5,0.092146,0.000498,Arden Heights,5,Staten Island,"POLYGON ((-74.17422 40.56257, -74.17349 40.562..."


In [8]:
zones.head()

,LocationID,Borough,Zone,service_zone
0,1,EWR,Newark Airport,EWR
1,2,Queens,Jamaica Bay,Boro Zone
2,3,Bronx,Allerton/Pelham Gardens,Boro Zone
3,4,Manhattan,Alphabet City,Yellow Zone
4,5,Staten Island,Arden Heights,Boro Zone


In [9]:
gdf = gpd.GeoDataFrame(
    pd.merge(zones, sf, on='LocationID', how='inner')
)

gdf.head()

,LocationID,Borough,Zone,service_zone,OBJECTID,Shape_Leng,Shape_Area,zone,borough,geometry
0,1,EWR,Newark Airport,EWR,1,0.116357,0.000782,Newark Airport,EWR,"POLYGON ((-74.18445 40.695, -74.18449 40.6951,..."
1,2,Queens,Jamaica Bay,Boro Zone,2,0.433470,0.004866,Jamaica Bay,Queens,"MULTIPOLYGON (((-73.82338 40.63899, -73.82277 ..."
2,3,Bronx,Allerton/Pelham Gardens,Boro Zone,3,0.084341,0.000314,Allerton/Pelham Gardens,Bronx,"POLYGON ((-73.84793 40.87134, -73.84725 40.870..."
3,4,Manhattan,Alphabet City,Yellow Zone,4,0.043567,0.000112,Alphabet City,Manhattan,"POLYGON ((-73.97177 40.72582, -73.97179 40.725..."
4,5,Staten Island,Arden Heights,Boro Zone,5,0.092146,0.000498,Arden Heights,Staten Island,"POLYGON ((-74.17422 40.56257, -74.17349 40.562..."


In [10]:
# create a JSON 
geoJSON = gdf[['LocationID', 'geometry']].drop_duplicates('LocationID').to_json()

# print the first 300 chars of the json
print(geoJSON[:300])

{"type": "FeatureCollection", "features": [{"id": "0", "type": "Feature", "properties": {"LocationID": 1}, "geometry": {"type": "Polygon", "coordinates": [[[-74.18445299999996, 40.694996000000096], [-74.18448899999999, 40.695095000000066], [-74.18449799999996, 40.69518500000006], [-74.18438099999997


Now we will visualise total amounts Uber earns, total tips drivers get, how long customers have to wait after request and how many total trips there are based on location ID

In [11]:
uber_df = pd.read_parquet("../data/curated/uber/uber.parquet")

In [12]:
uber_df

,dispatching_base_num,request_datetime,on_scene_datetime,pickup_datetime,dropoff_datetime,PULocationID,DOLocationID,trip_distance,trip_time,base_passenger_fare,tolls,bcf,sales_tax,congestion_surcharge,airport_fee,tips,driver_pay,total_amount,waiting_time,fare_per_miles
0,B03404,2023-08-07 23:59:24,2023-08-08 00:04:42,2023-08-08 00:06:42,2023-08-08 00:13:49,161,234,1.68,7.116667,23.44,0.0,0.64,2.08,2.75,0.0,0.00,13.16,28.91,5.300000,17.208333
1,B03404,2023-08-08 00:18:00,2023-08-08 00:23:37,2023-08-08 00:23:37,2023-08-08 00:35:34,188,61,3.11,11.950000,10.45,0.0,0.29,0.93,0.00,0.0,0.00,10.83,11.67,5.616667,3.752412
2,B03404,2023-08-08 00:48:41,2023-08-08 00:53:52,2023-08-08 00:55:00,2023-08-08 01:09:23,41,140,4.25,14.383333,12.82,0.0,0.35,1.14,2.75,0.0,0.00,13.96,17.06,5.183333,4.014118
3,B03404,2023-08-07 23:57:09,2023-08-07 23:58:19,2023-08-08 00:01:44,2023-08-08 00:16:12,230,246,2.07,14.466667,25.82,0.0,0.71,2.29,2.75,0.0,0.00,18.59,31.57,1.166667,15.251208
4,B03404,2023-08-08 00:15:24,2023-08-08 00:16:06,2023-08-08 00:17:41,2023-08-08 00:32:19,13,97,4.64,14.633333,35.59,0.0,0.98,3.16,2.75,0.0,8.49,24.21,50.97,0.700000,10.984914
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4067353,B03404,2023-07-31 23:32:33,2023-07-31 23:34:12,2023-07-31 23:35:25,2023-07-31 23:47:54,158,143,2.55,12.483333,53.18,0.0,1.46,4.72,2.75,0.0,0.00,27.66,62.11,1.650000,24.356863
4067354,B03404,2023-07-31 23:26:35,2023-07-31 23:31:37,2023-07-31 23:31:46,2023-07-31 23:53:50,89,225,4.58,22.066667,18.25,0.0,0.50,1.62,0.00,0.0,0.00,18.64,20.37,5.033333,4.447598
4067355,B03404,2023-07-31 23:17:46,2023-07-31 23:20:52,2023-07-31 23:22:53,2023-07-31 23:37:50,13,170,6.02,14.950000,27.44,0.0,0.75,2.44,2.75,0.0,3.29,16.34,36.67,3.100000,6.091362
4067356,B03404,2023-07-31 23:37:55,2023-07-31 23:39:52,2023-07-31 23:40:14,2023-07-31 23:54:32,42,168,3.76,14.300000,13.44,0.0,0.37,1.19,0.00,0.0,0.00,13.01,15.00,1.950000,3.989362


In [13]:
uber_df = uber_df \
    .merge(gdf[['LocationID', 'geometry']], left_on='PULocationID', right_on='LocationID') \
    .drop('LocationID', axis=1)

uber_df.head()

,dispatching_base_num,request_datetime,on_scene_datetime,pickup_datetime,dropoff_datetime,PULocationID,DOLocationID,trip_distance,trip_time,base_passenger_fare,...,bcf,sales_tax,congestion_surcharge,airport_fee,tips,driver_pay,total_amount,waiting_time,fare_per_miles,geometry
0,B03404,2023-08-07 23:59:24,2023-08-08 00:04:42,2023-08-08 00:06:42,2023-08-08 00:13:49,161,234,1.68,7.116667,23.44,...,0.64,2.08,2.75,0.0,0.0,13.16,28.91,5.300000,17.208333,"POLYGON ((-73.97534 40.7553, -73.97581 40.7546..."
1,B03404,2023-08-08 01:05:40,2023-08-08 01:06:06,2023-08-08 01:07:27,2023-08-08 01:27:43,161,167,7.22,20.266667,25.04,...,0.69,2.22,2.75,0.0,0.0,20.92,30.70,0.433333,4.252078,"POLYGON ((-73.97534 40.7553, -73.97581 40.7546..."
2,B03404,2023-08-08 01:00:22,2023-08-08 01:01:08,2023-08-08 01:01:15,2023-08-08 01:04:27,161,237,1.03,3.200000,8.22,...,0.23,0.73,2.75,0.0,0.0,5.39,11.93,0.766667,11.582524,"POLYGON ((-73.97534 40.7553, -73.97581 40.7546..."
3,B03404,2023-08-08 01:01:55,2023-08-08 01:07:30,2023-08-08 01:08:00,2023-08-08 01:29:26,161,127,9.11,21.433333,28.86,...,0.79,2.56,2.75,0.0,0.0,24.17,34.96,5.583333,3.837541,"POLYGON ((-73.97534 40.7553, -73.97581 40.7546..."
4,B03404,2023-08-08 01:15:45,2023-08-08 01:18:05,2023-08-08 01:20:05,2023-08-08 01:30:55,161,249,2.84,10.833333,18.42,...,0.51,1.63,2.75,0.0,0.0,9.97,23.31,2.333333,8.207746,"POLYGON ((-73.97534 40.7553, -73.97581 40.7546..."


In [14]:
# Total Amount
proportions = uber_df[['PULocationID', 'total_amount']] \
                .groupby('PULocationID') \
                .agg(
                    {
                        'total_amount': 'sum', # sum over total amount earned
                        'PULocationID': 'count' # count number of instances from sample
                    }
                ) \
                .rename({'PULocationID': 'total_trips'}, axis=1)

print(proportions.head())

proportions['proportion_of_total_amount'] = (
    # total amount earned from the Location ID * number of trips in Location / total number of trips
    proportions['total_amount'] * (proportions['total_trips'] / proportions['total_trips'].sum())
)
proportions = proportions.reset_index().sort_values('proportion_of_total_amount', ascending=False)

print(proportions.head())


m = folium.Map(location=[40.73, -73.74], tiles="Stamen Terrain", zoom_start=10)

# refer to the folium documentations on more information on how to plot aggregated data.
c = folium.Choropleth(
    geo_data=geoJSON, # geoJSON 
    name='choropleth', # name of plot
    data=proportions.reset_index(), # data source
    columns=['PULocationID', 'proportion_of_total_amount'], # the total_amountumns required
    key_on='properties.LocationID', # this is from the geoJSON's properties
    fill_color='YlOrRd', # color scheme
    nan_fill_color='black',
    legend_name='Proportion of total earnings USD$'
)

c.add_to(m)


gdf.loc[gdf['Zone'].str.contains('Airport')]
# (y, x) since we want (lat, long)
gdf['centroid'] = gdf['geometry'].apply(lambda x: (x.centroid.y, x.centroid.x))
gdf[['Zone', 'LocationID', 'centroid']].head()
for zone_name, coord in gdf.loc[gdf['Zone'].str.contains('Airport'), ['Zone', 'centroid']].values:
    m.add_child(
        folium.Marker(location=coord, popup=zone_name)
    )
m.save('../plots/uber/map/uber_total_earnings.html')


              total_amount  total_trips
PULocationID                           
2                   236.95            8
3                188112.53         8666
4                403995.93        13205
5                 33084.02         1061
6                 53663.06         1816
     PULocationID  total_amount  total_trips  proportion_of_total_amount
126           132    6866140.56        85126               143323.381308
132           138    5700900.34        79641               111332.582325
223           230    2614810.83        52718                33801.916906
155           161    2537966.64        54031                33625.677528
76             79    1884432.74        56320                26024.681081


In [15]:
# Total Tips
proportions = uber_df[['PULocationID', 'tips']] \
                .groupby('PULocationID') \
                .agg(
                    {
                        'tips': 'sum', # sum over total tips earned
                        'PULocationID': 'count' # count number of instances from sample
                    }
                ) \
                .rename({'PULocationID': 'total_trips'}, axis=1)

print(proportions.head())

proportions['proportion_of_tips'] = (
    # total tips earned from the Location ID * number of trips in Location / total number of trips
    proportions['tips'] * (proportions['total_trips'] / proportions['total_trips'].sum())
)
proportions = proportions.reset_index().sort_values('proportion_of_tips', ascending=False)

print(proportions.head())


m = folium.Map(location=[40.73, -73.74], tiles="Stamen Terrain", zoom_start=10)

# refer to the folium documentations on more information on how to plot aggregated data.
c = folium.Choropleth(
    geo_data=geoJSON, # geoJSON 
    name='choropleth', # name of plot
    data=proportions.reset_index(), # data source
    columns=['PULocationID', 'proportion_of_tips'], # the columns required
    key_on='properties.LocationID', # this is from the geoJSON's properties
    fill_color='YlOrRd', # color scheme
    nan_fill_color='black',
    legend_name='Proportion of tip earnings USD$'
)

c.add_to(m)


gdf.loc[gdf['Zone'].str.contains('Airport')]
# (y, x) since we want (lat, long)
gdf['centroid'] = gdf['geometry'].apply(lambda x: (x.centroid.y, x.centroid.x))
gdf[['Zone', 'LocationID', 'centroid']].head()
for zone_name, coord in gdf.loc[gdf['Zone'].str.contains('Airport'), ['Zone', 'centroid']].values:
    m.add_child(
        folium.Marker(location=coord, popup=zone_name)
    )
m.save('../plots/uber/map/uber_tip_earnings.html')


                  tips  total_trips
PULocationID                       
2                10.48            8
3              2943.46         8666
4             11840.35        13205
5              1854.87         1061
6              1877.11         1816
     PULocationID       tips  total_trips  proportion_of_tips
132           138  349386.79        79641         6823.156211
126           132  321417.95        85126         6709.257844
223           230  132181.17        52718         1708.718992
155           161  122633.42        54031         1624.777793
65             68   89257.21        47132         1031.576181


In [16]:
# Average Waiting Time
proportions = uber_df[['PULocationID', 'waiting_time']] \
                .groupby('PULocationID') \
                .agg(
                    {
                        'waiting_time': 'sum', # sum over waiting time
                        'PULocationID': 'count' # count number of instances from sample
                    }
                ) \
                .rename({'PULocationID': 'total_trips'}, axis=1)

print(proportions.head())

proportions['avg_waiting_time'] = (
    # average waiting time from the Location ID
    (proportions['waiting_time'] / proportions["total_trips"])
)
proportions = proportions.reset_index().sort_values('avg_waiting_time', ascending=False)

print(proportions.head())


m = folium.Map(location=[40.73, -73.74], tiles="Stamen Terrain", zoom_start=10)

# refer to the folium documentations on more information on how to plot aggregated data.
c = folium.Choropleth(
    geo_data=geoJSON, # geoJSON 
    name='choropleth', # name of plot
    data=proportions.reset_index(), # data source
    columns=['PULocationID', 'avg_waiting_time'], # the total columns required
    key_on='properties.LocationID', # this is from the geoJSON's properties
    fill_color='YlOrRd', # color scheme
    nan_fill_color='black',
    legend_name='Average waiting time'
)

c.add_to(m)


gdf.loc[gdf['Zone'].str.contains('Airport')]
# (y, x) since we want (lat, long)
gdf['centroid'] = gdf['geometry'].apply(lambda x: (x.centroid.y, x.centroid.x))
gdf[['Zone', 'LocationID', 'centroid']].head()
for zone_name, coord in gdf.loc[gdf['Zone'].str.contains('Airport'), ['Zone', 'centroid']].values:
    m.add_child(
        folium.Marker(location=coord, popup=zone_name)
    )
m.save('../plots/uber/map/uber_waiting_time.html')


              waiting_time  total_trips
PULocationID                           
2                47.950000            8
3             30878.350000         8666
4             52965.383333        13205
5              5833.300000         1061
6              7483.250000         1816
     PULocationID  waiting_time  total_trips  avg_waiting_time
25             27   3042.233333          386          7.881434
28             30   1806.433333          236          7.654379
104           110     81.566667           11          7.415152
188           194   5713.200000          795          7.186415
90             93  30096.933333         4447          6.767918


In [17]:
# Total Trips
m = folium.Map(location=[40.73, -73.74], tiles="Stamen Terrain", zoom_start=10)

# refer to the folium documentations on more information on how to plot aggregated data.
c = folium.Choropleth(
    geo_data=geoJSON, # geoJSON 
    name='choropleth', # name of plot
    data=proportions.reset_index(), # data source
    columns=['PULocationID', 'total_trips'], # the columns required
    key_on='properties.LocationID', # this is from the geoJSON's properties
    fill_color='YlOrRd', # color scheme
    nan_fill_color='black',
    legend_name='Proportion of trips'
)

c.add_to(m)


gdf.loc[gdf['Zone'].str.contains('Airport')]
# (y, x) since we want (lat, long)
gdf['centroid'] = gdf['geometry'].apply(lambda x: (x.centroid.y, x.centroid.x))
gdf[['Zone', 'LocationID', 'centroid']].head()
for zone_name, coord in gdf.loc[gdf['Zone'].str.contains('Airport'), ['Zone', 'centroid']].values:
    m.add_child(
        folium.Marker(location=coord, popup=zone_name)
    )
m.save('../plots/uber/map/uber_total_trips.html')

In [18]:
fare_per_miles = uber_df["total_amount"].sum() / uber_df["trip_distance"].sum()
fare_per_trips = uber_df["total_amount"].sum() / proportions["total_trips"].sum()
tips_per_trips = uber_df["tips"].sum() / proportions["total_trips"].sum()
tips_per_miles = uber_df["tips"].sum() / uber_df["trip_distance"].sum()
miles_per_trips = uber_df["trip_distance"].sum() / proportions["total_trips"].sum()
avg_waiting_time = uber_df["waiting_time"].sum() / proportions["total_trips"].sum()

print(f"Uber fare/miles: {fare_per_miles}")
print(f"Uber fare/trips: {fare_per_trips}")
print(f"Uber tips/trips: {tips_per_trips}")
print(f"Uber tips/miles: {tips_per_miles}")
print(f"Uber miles/trips: {miles_per_trips}")
print(f"Uber average waiting time: {avg_waiting_time}")

Uber fare/miles: 6.244620704012056
Uber fare/trips: 32.18293049459302
Uber tips/trips: 1.1562836688654017
Uber tips/miles: 0.22435970955227388
Uber miles/trips: 5.153704607537824
Uber average waiting time: 3.671791115960865


24/08/27 21:53:02 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors
24/08/28 01:37:49 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 306958 ms exceeds timeout 120000 ms
24/08/28 01:37:49 WARN SparkContext: Killing executors is not supported by current scheduler.
24/08/28 01:37:51 ERROR Inbox: Ignoring error
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.SparkThreadUtils$.awaitResult(SparkThreadUtils.scala:56)
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:310)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRefByURI(RpcEnv.scala:102)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRef(RpcEnv.scala:110)
	at org.apache.spark.util.Rpc